**Representing** words & numbers


In [18]:
%matplotlib inline

import string
from collections import Counter
from pprint import pprint
import gzip
import matplotlib.pyplot as plt
import numpy as np 


In [83]:
long_text = """It was the best of times, it was the worst of times, it was the age of wisdom, it was the age of foolishness, it was the epoch of belief, it was the epoch of incredulity, it was the season of Light, it was the season of Darkness, it was the spring of hope, it was the winter of despair, we had everything before us, we had nothing before us, we were all going direct to Heaven, we were all going direct the other way – in short, the period was so far like the present period, that some of its noisiest authorities insisted on its being received, for good or for evil, in the superlative degree of comparison only."""
short_text = """In fairy-tales, witches always wear silly black hats and black coats, and they ride on broomsticks. But this is not a fairy-tale. This is about REAL WITCHES."""
text = short_text

## Tokenization

In [5]:
def extract_words(text):
    temp = text.split()
    text_words = []

    for word in temp:
        # remove punctuation at beginning of word
        while word[0] in string.punctuation:
            word = word[1:]

        # remove punctuation at end of word
        while word[-1] in string.punctuation:
            word = word[:-1]

        # Append this word into our list of words
        text_words.append(word.lower())

    return text_words

In [6]:
text_words = extract_words(text)
print(text_words)

['in', 'fairy-tales', 'witches', 'always', 'wear', 'silly', 'black', 'hats', 'and', 'black', 'coats', 'and', 'they', 'ride', 'on', 'broomsticks', 'but', 'this', 'is', 'not', 'a', 'fairy-tale', 'this', 'is', 'about', 'real', 'witches']


## Motivation

ML algos tend not to work with categorical data. Rather than working with categorical data directly, **encoding** allows you to _more expressively_ represent the text (categorical) data. We do this by converting categories to numbers. 


## One Hot Encoding

1. is a more efficient way to represent vectors. 
2. the column feature vector defines a high dimensional space, where each dimension represents a word
3. each element is zero in this vector, except the element corresponding to the dimension representing the word
4. For _full-texts_ instead of words, the vector representation of the text is simply the vector sum of all the words it contains:



In [7]:
word_dict = {}
word_list = []
vocabulary_size = 0
text_tokens = []

for word in text_words:
    # create an ID for words seen for the first time & add to dictionary
    if word not in word_dict:
        word_dict[word] = vocabulary_size
        word_list.append(word)
        vocabulary_size += 1

    # add the token corresponding to the current word to the tokenized text.
    text_tokens.append(word_dict[word])

In [8]:
print("Word list:", word_list, "\n\n Word dictionary")
pprint(word_dict)

Word list: ['in', 'fairy-tales', 'witches', 'always', 'wear', 'silly', 'black', 'hats', 'and', 'coats', 'they', 'ride', 'on', 'broomsticks', 'but', 'this', 'is', 'not', 'a', 'fairy-tale', 'about', 'real'] 

 Word dictionary
{'a': 18,
 'about': 20,
 'always': 3,
 'and': 8,
 'black': 6,
 'broomsticks': 13,
 'but': 14,
 'coats': 9,
 'fairy-tale': 19,
 'fairy-tales': 1,
 'hats': 7,
 'in': 0,
 'is': 16,
 'not': 17,
 'on': 12,
 'real': 21,
 'ride': 11,
 'silly': 5,
 'they': 10,
 'this': 15,
 'wear': 4,
 'witches': 2}


In [9]:
print(text_tokens)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 6, 9, 8, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 15, 16, 20, 21, 2]


In [10]:
import re
text = """
Mary had a little lamb, little lamb,
little lamb, Mary had a little lamb
whose fleece was white as snow. 
And everywhere that Mary went
Mary went, Mary went, everywhere 
that Mary went
the lamb was sure to go
"""

In [11]:
text = re.sub(r'[^\w\s]', '', text) 
word_list = text.lower().split()

In [12]:
word_dict = {}
for word in word_list:
    if not word in word_dict.keys():
        word_dict[word] = 1
    else:
        word_dict[word] += 1

In [13]:
def one_hot(word, word_dict):
    """
    Generate a one-hot encoded vector for "word"
    """

    vector = np.zeros(len(word_dict))
    vector[word_dict[word]] = 1
    return vector

    

In [14]:
fleece_hot = one_hot('fleece', word_dict)
print(fleece_hot)

[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [15]:
mary_hot = one_hot('mary', word_dict)
print(mary_hot)

[0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [16]:
mary_hot[6] == 1

True

## OHE with Scikit-learn

1. Pre-process (transform) the data
    a. Lower case
    b. Split by space
2. Integer encode using `fit_transform` on the dataset
3. Alternatively, use binary encoding

`LabelEncoder` can be used to normalize labels


In [84]:
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder # Try ?LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [94]:
# 1. convert corpus documents to lower case
doc1 = long_text.lower()
doc2 = short_text.lower()

# convert to unary tokens
doc1 = doc1.split()
doc2 = doc2.split()
dataset = array(doc1+doc2) # convert the list to a numpy array of string

In [108]:
# 2. Integer encode the dataset
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(dataset) # alphabetically assign integers to each word
print(integer_encoded)
print(integer_encoded.shape)

[38 70 64 10 45 67 38 70 64 79 45 67 38 70 64  2 45 76 38 70 64  2 45 26
 38 70 64 20 45  9 38 70 64 20 45 35 38 70 64 56 45 40 38 70 64 56 45 16
 38 70 64 61 45 33 38 70 64 75 45 18 72 30 21  7 69 72 30 44  7 69 72 74
  3 28 19 68 32 72 74  3 28 19 64 49 71 80 34 57 64 50 70 59 25 41 64 52
 51 63 60 45 39 42  6 36 46 39  8 54 27 29 48 27 22 34 64 62 17 45 15 47
 34 24 77  4 73 58 11 31  5 11 14  5 65 55 46 12 13 66 37 43  0 23 66 37
  1 53 78]
(147,)


In [109]:
max(integer_encoded)

80

In [110]:
# 3. Binary encode
onehot_encoder = OneHotEncoder(sparse=False)

## Reshape `integer_encoded` into a 2D array required for `OneHotEncoder` instance
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded) # convert to an array of size integer_encoded x max(integer_encoded)
print(onehot_encoded[0,:]) 

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [114]:
# invert the first word of long_text
inverted = label_encoder.inverse_transform([argmax(onehot_encoded[0, :])])
print(inverted)
print(doc1[0])

['it']
it
